# Introduction

At first we decided to download the database from the [MIMIC-Website](https://mimic.physionet.org/gettingstarted/cloud/) and work with the csv-Files. But after try to read some huge files (>1GB) it is necessary to use a own database. Due to the fact that we work as a team, we decided to install a database in the Google Cloud and set up access for the team members

# Creating the database

*Future work how to create the database in GCP*

# Connect to the database

## Unix (mac) - prework

To use the GCP-Database from our Computer, we need to install some software vom Google.

### Install Google Cloud SDK

You can find the online documentation here: https://cloud.google.com/sdk/docs/quickstarts

For macOS use the terminal an download the source: <br>
`curl -O https://dl.google.com/dl/cloudsdk/channels/rapid/downloads/google-cloud-sdk-308.0.0-darwin-x86_64.tar.gz`

Unzip the zip.file at preferred storage location. Here our home path <br>
`gunzip google-cloud-sdk-308.0.0-darwin-x86_64.tar.gz`

Use install script (check the path of your SDK-Installation:<br>
`./google-cloud-sdk/install.sh` 

Start the Google Cloud SDK: <br>
`./google-cloud-sdk/bin/gcloud init`

After start the Cloud SDK, you have to accept the option to log in using your Google user account. Follow the instruction from the [documentation](https://cloud.google.com/sdk/docs/quickstart#initializing_the) <br>
For this part here, the last point is normally 4.  
Check the result with `gcloud auth list` 

### Install Google Cloud Proxy

The proxy manage the connection to the database without handling certificates and stuff. It simulate a lokal database. [Link](https://cloud.google.com/sql/docs/postgres/connect-external-app#proxy)

First download the source: <br>
`curl -o cloud_sql_proxy https://dl.google.com/cloudsql/cloud_sql_proxy.darwin.amd64`

Make the proxy executable: <br>
`chmod +x cloud_sql_proxy`

### Start the Proxy

To use the database, it is necessary to start ot initialize the proxy with:<br>
`./cloud_sql_proxy -instances=[INSTANCE_CONNECTION_NAME]:capstone=tcp:5444 &`

We don't use the default port for postgresSQL at the proxy, if someone has a local database instance running.

## Python

If not already installed: <br>
`conda install pyscopg2` 

In [ ]:
import pandas as pd
import csv
import psycopg2

# settings
sql_path = './sql/'
pd.set_option('display.max_columns', 500)

# database
conn_string = open('./intern/DB_connection.txt', 'r', encoding='UTF-8').read()

In [ ]:
# Connect to Database
conn = eval('psycopg2.connect({})'.format(conn_string.strip()))

In [ ]:
# Check connection
sql = """select * from admissions limit 1000"""
df = pd.read_sql(sql, conn)

In [ ]:
df.head(3)

,ROW_ID,SUBJECT_ID,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,LANGUAGE,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,21,22,165315,2196-04-09 14:26:00,2196-04-10 17:54:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 12:06:00,2196-04-09 15:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 09:15:00,2153-09-08 21:10:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,,CATHOLIC,MARRIED,WHITE,NaT,NaT,CORONARY ARTERY DISEASECORONARY ARTERY BYPASS ...,0,1
2,23,23,124321,2157-10-18 21:34:00,2157-10-25 16:00:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaT,NaT,BRAIN MASS,0,1


In [ ]:
# Close connection
conn.close()